
# 3D Darcy flow

Here we illustrate Darcy flow in a heterogeneous 3D body. We use the general
:py:func:`pygimli.solver.solveFiniteElements` to solve Darcy's law:

$\begin{align}\nabla \cdot(K \nabla p)=0\end{align}$

The sought hydraulic velocity distribution can then be calculated as the
gradient field of $\mathbf{v}=-\nabla p$.


In [18]:
import numpy as np
import pygimli as pg
import pygimli.meshtools as mt
import pyvista
from pygimli.viewer.pv import drawStreamLines, drawSlice

#### Create Mesh

In [19]:
plc = mt.createCube(size=[40, 20, 15], marker=1, boundaryMarker=0)
cube = mt.createCube(size=[15, 15, 8], marker=2, boundaryMarker=0)
geom = plc + cube

mesh = mt.createMesh(geom, area=4)

#### Set Boundary Conditions

In [20]:
for bound in mesh.boundaries():
    x = bound.center().x()
    if x == mesh.xmin():
        bound.setMarker(1)
    elif x == mesh.xmax():
        bound.setMarker(2)


bc = {"Dirichlet": {1: 20.0, 2: 10.0}}



#### Define permeability

In [27]:
kMap = {1: 1e-4, 2: 1e-6}
kArray = pg.solver.parseMapToCellArray(list(kMap), mesh) # dict does not work
kArray2 = np.column_stack([kArray] * 3)

#### Solve the equations

In [22]:
h = pg.solver.solveFiniteElements(mesh, kMap, bc=bc)
vel = -pg.solver.grad(mesh, h) * kArray
pg.show(mesh, h, label="Hydraulic head (m)")

(<pyvista.plotting.plotting.Plotter at 0x2664b56ebb0>, None)

In [26]:
ax, _ = pg.show(mesh, alpha=0.3, hold=True, colorBar=False)
drawStreamLines(ax, mesh, vel, radius=.1, source_radius=10)
drawSlice(ax, mesh, normal=[0,1,0], data=pg.abs(vel), label="Absolute velocity")
# ax.show()